In [1]:
import joblib
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

import configuration
from notebook_utils import notebookToPython
from label_reader import readLabels

In [29]:
config = configuration.ConfigLoader('config.yml')

notebookToPython(config.projectName)
job = config.getJobConfig(config.activeJobId)

Write python file


In [3]:
trainingLabels = readLabels(job)

Loading C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt...


In [4]:
X = []
y = []


fullDataPath = job.fullJoinFilePath(job.dataPath, job.trainingDataPath)

for filename, label in trainingLabels.items():
    audioSourceFilename = job.fullJoinFilePath(fullDataPath, filename + job.trainingDataExtension)
    
    audio, _ = librosa.load(audioSourceFilename, sr = job.sampleRate, duration = job.duration)

    mel_spectrogram = librosa.feature.melspectrogram(y = audio, sr = job.sampleRate, n_mels = job.numMels)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

    if (mel_spectrogram.shape[1] < job.maxTimeSteps):
        padWidth = ((0, 0), (0, job.maxTimeSteps - mel_spectrogram.shape[1]))
        mel_spectrogram = np.pad(array=mel_spectrogram, pad_width=padWidth, mode='constant')
    else:
        mel_spectrogram = mel_spectrogram[:, :job.maxTimeSteps]

    X.append(mel_spectrogram)
    y.append(label)

In [5]:
X = np.array(X)
y = np.array(y)
y_encoded = to_categorical(y, job.numClasses)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2)    # test data is 20% of all data

In [13]:
# Define CNN model architecture
input_shape = (job.numMels, X_train.shape[2], 1)  # Input shape for CNN (height, width, channels)
model_input = Input(shape=input_shape)


In [14]:
# TODO - why were these parameters selected? What purpose do they serve? Should they be configurable?
x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(model_input)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(0.5)(x)

model_output = Dense(job.numClasses, activation='softmax')(x)

In [15]:
model = Model(inputs=model_input, outputs=model_output)


In [16]:
model.compile(optimizer=job.optimizer, loss=job.loss, metrics=job.metrics)

In [17]:
# Train the Model
model.fit(X_train, y_train, batch_size=job.batchSize, epochs=job.numEpochs, validation_data=(X_test, y_test))

Epoch 1/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 43s 65ms/step - accuracy: 0.8952 - loss: 1.5125 - val_accuracy: 0.9070 - val_loss: 0.1970
Epoch 2/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 40s 64ms/step - accuracy: 0.9288 - loss: 0.1756 - val_accuracy: 0.9748 - val_loss: 0.0702
Epoch 3/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 40s 63ms/step - accuracy: 0.9694 - loss: 0.0831 - val_accuracy: 0.9817 - val_loss: 0.0467
Epoch 4/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 40s 63ms/step - accuracy: 0.9807 - loss: 0.0512 - val_accuracy: 0.9854 - val_loss: 0.0430
Epoch 5/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 41s 64ms/step - accuracy: 0.9896 - loss: 0.0354 - val_accuracy: 0.9854 - val_loss: 0.0372
Epoch 6/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 41s 64ms/step - accuracy: 0.9894 - loss: 0.0256 - val_accuracy: 0.9829 - val_loss: 0.0455
Epoch 7/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 41s 65ms/step - accuracy: 0.9885 - loss: 0.0328 - val_accuracy: 0.9915 - val_loss: 0.0317
Epoch 8/10
635/635 ━━━━━━━━━━━━━━━━━━━━ 44s 69ms/step - accuracy: 0.9941 - loss: 0.0180 - 

In [18]:
joblib.dump(model, job.persistedModel)

['ASVspoof-2019-1.libjob']

### Test Model

In [19]:
import joblib
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

import configuration
from notebook_utils import notebookToPython
from label_reader import readLabels

In [20]:
config = configuration.ConfigLoader('config.yml')

notebookToPython(config.projectName)
job = config.getJobConfig(config.activeJobId)

Write python file


In [21]:
model = joblib.load(job.persistedModel)

In [26]:
# y_pred=model.predict(X_test) 
# y_pred=np.argmax(y_pred, axis=1)
# y_test=np.argmax(y_test, axis=1)
# cm = confusion_matrix(y_test, y_pred)
# print(cm)
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
y_pred

159/159 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step


array([0, 0, 0, ..., 0, 0, 0])

In [27]:
y_test

array([0, 0, 0, ..., 0, 0, 0])

In [28]:
score = accuracy_score(y_test, y_pred)
score

0.9948778565799843